# Extrae Las preguntas de los PDFs

Este notebook extrae la preguntas de los PDF previamente existentes en una ubicación dada.



## Entorno

### Librerías

In [ ]:
from logutils import get_logger
from exam_question_downloader.utils.consts import ROOT_DIR
from pathlib import Path
import pickle
from exam_question_downloader.utils.text_processing import get_text_docs_from_dir, extract_questions_from_doc, classify_questions_by_subject
from exam_question_downloader.utils.text_processing import get_text_docs_from_file, get_questions_not_classified
from exam_question_downloader.utils.text_formating import format_question, format_questions

In [ ]:
# SOLO DURANTE EL DESARROLLO; Recarga el código fuente de las librerías en cada ejecucion de celda
%load_ext autoreload
%autoreload 2

In [ ]:
logger = get_logger("YEA")

### Variables

In [ ]:
# Ubicacion de los PDF a procesar
INPUT_PATH=ROOT_DIR.joinpath("..").joinpath("material-opos").joinpath("PDFsExamenes").joinpath("2024-04-ExamenA1Multimedia")

print(f"Directorio de los exámenes a analizar: {INPUT_PATH}")

In [ ]:
# Fichero de salida de los PDF
OUTPUT_PATH=ROOT_DIR.joinpath("..").joinpath("material-opos").joinpath("CSVsExamenes").joinpath("2024-04-ExamenA1Multimedia")
OUTPUT_PATH.mkdir(exist_ok=True)

print(f"Los CSV se guardarán en: {OUTPUT_PATH}")

## Leer PDFs

In [ ]:
# Buscar archivos PDF en el directorio dado de forma recursiva
archivos_pdf = list(INPUT_PATH.rglob('*.pdf'))

# Mostrar los archivos PDF encontrados
for archivo in archivos_pdf:
    print(archivo)

In [ ]:
# Cargar en un pickle el texto de todos los PDF
textdocs = []

for archivo in archivos_pdf:
    text_doc = get_text_docs_from_file(archivo)
    textdocs.append((archivo.name, text_doc))

textdocs

In [ ]:
# Detectar las preguntas y guardarlos en otro picke
questiondocs = []
for name, doc in textdocs:
    questiondocs.append((name, extract_questions_from_doc(doc)))

questiondocs

In [ ]:
# Estadísticas de las preguntas reconocidas y leídas;

totalpreguntas=0
for doc, preguntas in questiondocs:
    numero_preguntas = len(preguntas)
    totalpreguntas=totalpreguntas+numero_preguntas
    print(f"El archivo '{doc}' tiene {numero_preguntas} preguntas.")

print(f"En total se han leído {totalpreguntas} preguntas.")

In [ ]:
# Leemos las preguntas y las clasificamos por temas
df = classify_questions_by_subject(questiondocs)
df

In [ ]:
# Estadísticas de la clasificación de preguntas
print (f"De un total de {totalpreguntas} se han clasificado {df.shape[0]} preguntas.")

conteo = df['subject'].value_counts()
print(conteo)



## Preguntas no clasificadas en temas

In [ ]:
# Las preguntas que no se han clasificado
df2 = get_questions_not_classified(questiondocs)

df2

In [ ]:
print(format_question(df2.iloc[0]))

In [ ]:
print(format_questions(df2))


## Exportación de las preguntas clasificadas al directorio de salida


In [ ]:
from datetime import datetime

now = datetime.now().strftime("%Y-%m-%d")
filename=now+"-preguntas.csv"
filepath= OUTPUT_PATH.joinpath(filename)

print(f"Nombre del fichero de salida: {filename}")
print(f"Ruta del fichero de salida: {filepath}")

In [ ]:
# Exportación
df.to_csv(filepath, index=False)

In [ ]:
# Guardamos un informe de la exportación

# Nombre del fichero
filename=now+"-informe.txt"
filepath= OUTPUT_PATH.joinpath(filename)

# Abrir el fichero en modo escritura
with open(filepath, 'w') as file:

    totalpreguntas=0
    for doc, preguntas in questiondocs:
        numero_preguntas = len(preguntas)
        totalpreguntas=totalpreguntas+numero_preguntas
        print(f"El archivo '{doc}' tiene {numero_preguntas} preguntas.")
        file.write(f"El archivo '{doc}' tiene {numero_preguntas} preguntas.\n")

    print(f"En total se han leído {totalpreguntas} preguntas.")
    file.write(f"En total se han leído {totalpreguntas} preguntas.\n")

    print(f"Del total de {totalpreguntas} se han clasificado {df.shape[0]} preguntas.")
    file.write(f"Del total de {totalpreguntas} se han clasificado {df.shape[0]} preguntas.\n")